In [213]:
import pandas as pd

# Path to the CSV file
file_path = r'C:\Users\Titouan\Desktop\Github\Paren\data\evsession_095_data_20240131.csv'


def convert_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    return data

def filter_data(data, month = 1 , cities = ['San Jose', 'Denver', 'Chicago']):
    
    # Convert 'last_updated' from Unix timestamp to datetime (if your column is named differently, change accordingly)
    data['date'] = pd.to_datetime(data['date'])
    
    # Filter for the specific cities
    city_filter = data['city'].isin(cities)
    
    # Filter for January
    january_filter = data['date'].dt.month == month
    
    # Apply filters
    filtered_data = data[city_filter & january_filter]
    
    return filtered_data

filtered_data = filter_data(convert_csv(file_path))


Above we created 2 functions the first simply reads the csv file and converts it to a pandas Dataframe, the second filters the data by date and city.
We could have continued using all the data at first but I chose to work with the filtered data as it is a lot less memory heavy and reduces the run time of my algorithms as it takes multiple seconds on my computer to do the filter operation.


Our objective is to analyze the charging session and failure data from the 3 cities, Chicago,IL,  Denver,CO, and San Jose, CA for the month of January.
We want to know over what time period was there a large deviation in charger performance? And in what city was the change the most noticeable?
Finally we will try to find what other data we could use to correlate our findings?

To awnser the first  part of this problem I created performance scores:
    1. success rate which represents the success rate of charging sessions 
    2. avg charge time which as stated looks at the average charge time of succesful sessions
    3. retry rate which measures the proportion of succesful sessions with retries over the total amount of succesful sessions
    4. replug_ccs which represents the proportion of ccs sessions with replugs with retries over the total amount of succesful ccs sessions
    5. short session proportion which represents the number of short sessions divided by the total amount of sessions
    6. Charger availability caluclates the proportion of time the charger is available and not infered as offline or detected offline

After creating these indicators we want to try to observe a deviation to do so we will measure there moving average and moving standard deviation over time.


Below I created multiple different functions that will help me plot these:

    calculate_metrics will output the standard deviation and the average of these performance indicator for each date

    filter_significant_deviations will filter the rows that have a high deviation either from the mean 

    calculate_metrics_avg_rolling outputs the rolling average of the data given

    both plot metrics and finalize plot are quite specific and simplify the plotting



In [337]:
import matplotlib.pyplot as plt
import numpy as np


def calculate_metrics_avg_rolling(data,columns = ['success_rate', 'avg_charge_time', 'retry_rate', 'replug_rate_ccs', 'availability', 'short_session_proportion'], window_size=7):

    # Ensure data is sorted by date
    data = data.sort_values('date')

    # Calculate daily mean
    daily_mean = data.groupby('date')[columns].mean()
    daily_std = data.groupby('date')[columns].std()

    # Initialize a DataFrame to hold the rolling statistics
    rolling_stats = pd.DataFrame(index=daily_mean.index)

    # Calculate rolling means and standard deviations using the full dataset
    for column in columns:
        # Calculate rolling mean and standard deviation of daily means
        rolling_mean = daily_mean[column].rolling(window=window_size, min_periods=1).mean()
        if daily_std[column].isna().all(): rolling_std = daily_mean[column].rolling(window=window_size, min_periods=1).std()
        else: rolling_std = daily_std[column].rolling(window=window_size, min_periods=1).mean()

        # Assign to rolling stats DataFrame
        rolling_stats[f'{column}_rolling_avg'] = rolling_mean
        rolling_stats[f'{column}_rolling_std'] = rolling_std

    # Reset the index and include the date
    rolling_stats.reset_index(inplace=True)

    return rolling_stats



def calculate_metrics(data, city=None, station=None, window_size = 7):
    if city:
        data = data[data['city'] == city].copy()
    if station:
        data = data[data['name'] == station].copy()

    data.loc[:, 'total_success'] = data['charges_ccs'] + data['charges_cha']
    data.loc[:, 'total_failures'] = data['fails_ccs'] + data['fails_cha']
    data.loc[:, 'total_sessions'] = data['total_success'] + data['total_failures']

    # Prevent division by zero by adding small epsilon where total_sessions is zero
    data.loc[:, 'success_rate'] = np.where(data['total_sessions']>0,data['total_success'] / (data['total_sessions']),0)
    data.loc[:, 'avg_charge_time'] = (data['avg_charge_ccs'] * data['charges_ccs'] + data['avg_charge_cha'] * data['charges_cha']) / (data['total_success'] + np.finfo(float).eps)
    data.loc[:, 'retry_rate'] = (data['retries_ccs'] + data['retries_cha']) / (data['total_success'] + np.finfo(float).eps)
    data.loc[:, 'replug_rate_ccs'] = data['replugs_ccs'] / (data['charges_ccs'] + np.finfo(float).eps)
    data.loc[:, 'availability'] = data['open_secs'] / (data['open_secs'] + data['detected_dt_p1'] + data['detected_dt_p2'] + data['inferred_dt_p1'] + data['inferred_dt_p2'] + np.finfo(float).eps)

    short_session_columns = [f'bin_{i}_ccs' for i in range(1, 10)] + [f'bin_{i}_cha' for i in range(1, 10)]
    data['short_session_total'] = data[short_session_columns].sum(axis=1)
    long_session_columns = [f'bin_{i}_ccs' for i in [10, 20, 30, 40, 50, 60, 75, 90, 120]] + [f'bin_{i}_cha' for i in [10, 20, 30, 40, 50, 60, 75, 90, 120]]
    data['long_session_total'] = data[long_session_columns].sum(axis=1)

    data.loc[:, 'short_session_proportion'] = data['short_session_total'] / (data['short_session_total'] + data['long_session_total'] + np.finfo(float).eps)


    
    daily_averages = calculate_metrics_avg_rolling(data,window_size=window_size)

    return daily_averages


def filter_significant_deviations(data, column='success_rate', window_size=7, threshold=2, outlier_type='all'):
    """
    Filters significant deviations based on a specified column and criteria,
    without modifying the original DataFrame.
    """
    # Create a copy of the data to work on
    working_data = data.copy()

    # Compute new columns based on the original data
    working_data['total_success'] = working_data['charges_ccs'] + working_data['charges_cha']
    working_data['total_failures'] = working_data['fails_ccs'] + working_data['fails_cha']
    working_data['total_sessions'] = working_data['total_success'] + working_data['total_failures']

    if column == 'success_rate':
        working_data['success_rate'] = np.where(working_data['total_sessions'] > 0,
                                                working_data['total_success'] / (working_data['total_sessions']), 0)
    if column == 'avg_charge_time':
        working_data['avg_charge_time'] = (working_data['avg_charge_ccs'] * working_data['charges_ccs'] + working_data['avg_charge_cha'] * working_data['charges_cha']) / (working_data['total_success'] + np.finfo(float).eps)
    if column == 'retry_rate':
        working_data['retry_rate'] = (working_data['retries_ccs'] + working_data['retries_cha']) / (working_data['total_success'] + np.finfo(float).eps)
    if column == 'replug_rate_ccs':
        working_data['replug_rate_ccs'] = working_data['replugs_ccs'] / (working_data['charges_ccs'] + np.finfo(float).eps)
    if column == 'availability':
        working_data['availability'] = working_data['open_secs'] / (working_data['open_secs'] + working_data['detected_dt_p1'] +
                                                                    working_data['detected_dt_p2'] + working_data['inferred_dt_p1'] +
                                                                    working_data['inferred_dt_p2'] + np.finfo(float).eps)
    if column == 'short_session_proportion':
        short_session_columns = [f'bin_{i}_ccs' for i in range(1, 10)] + [f'bin_{i}_cha' for i in range(1, 10)]
        working_data['short_session_total'] = working_data[short_session_columns].sum(axis=1)
        long_session_columns = [f'bin_{i}_ccs' for i in [10, 20, 30, 40, 50, 60, 75, 90, 120]] + \
                               [f'bin_{i}_cha' for i in [10, 20, 30, 40, 50, 60, 75, 90, 120]]
        working_data['long_session_total'] = working_data[long_session_columns].sum(axis=1)
        working_data['short_session_proportion'] = working_data['short_session_total'] /(working_data['short_session_total'] + working_data['long_session_total'] + np.finfo(float).eps)

    # Ensure data is sorted by date
    working_data = working_data.sort_values('date')
    # Calculate daily mean
    working_data['daily_mean'] = working_data.groupby('date')[column].transform('mean')

    # Calculate daily standard deviation and broadcast it back to the original DataFrame's shape
    working_data['daily_std'] = working_data.groupby('date')[column].transform('std')           


    # Calculate rolling mean and standard deviation of daily means
    working_data[f'{column}_rolling_mean']= working_data['daily_mean'].rolling(window=window_size, min_periods=1).mean()
    
    working_data[f'{column}_rolling_std'] = working_data['daily_std'].rolling(window=window_size, min_periods=1).mean()


    # Calculate the upper and lower bounds
    working_data['upper_bound'] = working_data[f'{column}_rolling_mean'] + (threshold * working_data[f'{column}_rolling_std'])
    working_data['lower_bound'] = working_data[f'{column}_rolling_mean'] - (threshold * working_data[f'{column}_rolling_std'])

    

    # Create masks based on the outlier detection method
    mask = None
    if outlier_type == 'not':
        mask = (working_data[column] <= working_data['upper_bound']) & (working_data[column] >= working_data['lower_bound'])
    elif outlier_type == 'all':
        mask = (working_data[column] > working_data['upper_bound']) | (working_data[column] < working_data['lower_bound'])
    elif outlier_type == 'low':
        mask = working_data[column] < working_data['lower_bound']
    else:  # Assuming 'high'
        mask = working_data[column] > working_data['upper_bound']

    # Apply the mask to the original data and return modified data
    data = data.loc[mask]
    return data



def plot_metrics(data, axs, label_prefix,color):
    metrics = ['success_rate', 'avg_charge_time', 'retry_rate', 'replug_rate_ccs', 'availability', 'short_session_proportion']
    metric_names = ['Success Rate', 'Charge Time', 'Retry Rate', 'Replug Rate','Short Session Proportion','Availability']
    for i, metric in enumerate(metrics):
        axs[2*i].plot(data['date'], data[f'{metric}_rolling_avg'], label=f'Avg {metric_names[i]} - {label_prefix}', color=color)
        axs[2*i + 1].plot(data['date'], data[f'{metric}_rolling_std'], label=f'Std Dev {metric_names[i]} - {label_prefix}', color=color)

def finalize_plots(axs):
    for ax in axs:
        ax.set_xlabel('Date')
        ax.legend()
    axs[0].set_ylabel('Success Rate')
    axs[2].set_ylabel('Charge Time')
    axs[4].set_ylabel('Retry Rate')
    axs[6].set_ylabel('Replug Rate')
    axs[8].set_ylabel('Short Session Proportion')
    axs[10].set_ylabel('Availability')
    plt.tight_layout()
    plt.show()

def plot_metrics_std_avg_multi(data, window_size=7, cities=None, stations=None, filter = False, threshold = 2, filter_column = "success_rate",outlier_type = 'all'):
    fig, axs = plt.subplots(12, 1, figsize=(10, 40))

    colors = ['blue', 'red', 'green', 'magenta', 'yellow', 'orange', 'purple', 'pink',
        'lime', 'cyan', 'brown', 'gray', 'olive', 'maroon', 'navy', 'teal',
        'aqua', 'coral', 'fuchsia', 'gold', 'indigo', 'khaki', 'lavender',
        'plum', 'salmon', 'tan', 'violet', 'azure', 'beige', 'bisque',]

    i = 0

    if cities and len(cities) == 1 and not stations:
        city = cities[0]
        data = filter_data(data,cities=cities)
        if filter : filt_data = filter_significant_deviations(data,filter_column, window_size ,threshold,outlier_type)
        stations = filt_data[filt_data['city'] == city]['name'].unique()
        for station in stations:
            station_data = calculate_metrics(data, city, station, window_size = window_size)
            plot_metrics(station_data, axs, station,colors[i])
            i+=1
    elif cities:
        for city in cities:
            city_data = calculate_metrics(data, city, window_size = window_size)
            plot_metrics(city_data, axs, city,colors[i])
            i+=1
    elif not cities:
        general_data = calculate_metrics(data, window_size = window_size)
        plot_metrics(general_data, axs, 'total',colors[i])
        i+=1
    elif stations:
        for station in stations:
            station_data = calculate_metrics(data, cities[0], station, window_size = window_size)
            plot_metrics(station_data, axs, station,colors[i])
            i+=1

    finalize_plots(axs)


In [ ]:
plot_metrics_std_avg_multi(filtered_data, cities=['San Jose','Denver','Chicago'])

In [ ]:
plot_metrics_std_avg_multi(filtered_data,window_size= 2, cities=['San Jose','Denver','Chicago'])

Analysis of plots:

We have generated plots for three different rolling windows: 7 days, 4 days, and 2 days. In our initial analysis, we noted that San Jose shows no significant performance deviations compared to Denver and Chicago.

Significantly, between the 13th and 20th, Denver and Chicago experienced a notable decrease in average success rates and availability, and a corresponding increase in average retry rates. While the average charge times remained consistent, there was a noticeable increase in the variability of charge times. This suggests that specific stations in these cities may be encountering issues that are affecting performance. Additionally, a slight increase in the standard deviation of both the success rate and retry rate further indicates potential problems at these stations
For the short sessions only Chicago is victim of a measurable change in those dates for both average and deviation.

It also seems that the increase in standard deviation in charge time in Denver happens a bit earlier though a second peak is observed at the same time as the one in Chicago. Further analysis is necessary to understand why this might be the case. We will try to find the outlier charging stations. 

Finally, it seems clear that Chicago is more affected by this issue then both Denver and San Jose.

Additional data could be used to correlate these findings, such as city power load and power production (even down to a lower granularity like bus data). We could also correlate the data with outside temperature and weather conditions. Additionally, it would be interesting to see if the underperformance aligns with specific days linked to store promotions, holidays, or football games, which could have caused a surge in demand.

In [ ]:
plot_metrics_std_avg_multi(filtered_data,cities=['Chicago'],filter= True,threshold=2.8, outlier_type='low')

In [ ]:
plot_metrics_std_avg_multi(filtered_data,filter_column='retry_rate',cities=['Chicago'],filter= True,threshold=1, outlier_type='low')

We utilize the filter option in our function to highlight stations that show a significant deviation from the rolling mean of different metrics. This approach aids in identifying problematic substations. Our initial observations indicate that stations in Chicago have experienced notable issues during the specified time period. However, the data appears noisy, making it challenging to pinpoint underperforming stations due to the presence of generally underperforming stations. This observation suggests the need for a more sophisticated algorithm to accurately identify these stations.

In the future we could use other ML based outlier detection techniques like PCA adapted to timeseries or transformer based algorithms to identify the problematic substations. 